## Kaggle 대회 정리

### 우주왕복선 타이타닉 : 어떤 탑승객이 다른차원으로 전송되었는지 예측하라!

* 대회명 : Spaceship Titanic : Predict which passengers are transported to an alternate dimension
* 링크 : https://www.kaggle.com/competitions/spaceship-titanic

**<목차>**
1. 대회이해하기
2. EDA\#01 : 데이터체크
3. EDA\#02 : 데이터 이해(범주형)
3. EDA\#03 : 데이터 이해(연속형)
4. **모델링#01 : 벤치마크 모델 만들기**

In [27]:
# 라이브러리 로드

# 시스템 라이브러리

# Error가 아닌 Warnning 숨김 (에러가 아닌데 버전변경됐다 정도 메시지가 붉은색으로 나와 보기 싫음)
import warnings
warnings.filterwarnings('ignore')

# 데이터 핸들링 라이브러리
import pandas as pd
import numpy as np

#시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns


from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

# matplolib 한글폰트 랜더링을 위한 font설정
plt.rc('font', family='AppleGothic') # MacOS
plt.rc('font', size=9)  

%matplotlib inline

### STEP01 : 데이터 로드

In [10]:
## 경로설정
path_train = "./raw/train.csv"
path_test = "./raw/test.csv"
path_submission = "./raw/sample_submission.csv"


## pandas DataFrame으로 로드
raw_train = pd.read_csv(path_train)
raw_test = pd.read_csv(path_test)
raw_submission = pd.read_csv(path_submission)

In [11]:
raw_train.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


### 전처리

* 결측값 처리 : 제거
* 범주형 변수 라벨 인코딩
* 피쳐선택(Feature selection) : HomePlanet, CryoSleep, Destination, Age, VIP, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
* KNN은 귀찮게도 코드화를 해줘야함.

In [51]:
# Nan 값 모두 제거(결측치가 있으면 모델 학습이 안됨.)
pre_train = raw_train.dropna()

In [52]:
# 범주형 변수 라벨 인코딩
pre_train["preHomePlanet"] = pre_train["HomePlanet"].astype('category').cat.codes
pre_train["preCryoSleep"] = pre_train["CryoSleep"].astype('category').cat.codes
pre_train["preDestination"] = pre_train["Destination"].astype('category').cat.codes

In [56]:
# 변수선택 후 모델 학습을 위한 데이터생성
X = pre_train[["preHomePlanet", "preCryoSleep", "preDestination", "Age", "VIP"
               , "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]]

y = pre_train["Transported"]

# 모델을 학습할 데이터인 중 80%와 테스트할 20%를 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

In [57]:
# estimater 생성 (모델 생성, k=1)
clf = neighbors.KNeighborsClassifier(n_neighbors=5)

# 모델 훈련(train data)(보통 feeding 이라고 부름: 데이터를 먹인다)
clf.fit(X_train, y_train)

KNeighborsClassifier()

In [58]:
pred = clf.predict(X_test)

In [59]:
accuracy_score(pred,y_test)

0.7708018154311649